# Expense Data Analysis

Comprehensive analysis of expense data for insights and patterns.

## Objectives:
1. Load and explore expense data
2. Analyze spending patterns
3. Identify trends and seasonality
4. Detect anomalies
5. Generate actionable insights

In [ ]:
# Import libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

from backend.models.database import SessionLocal
from backend.models import tables
from backend.config.constants import EXPENSE_CATEGORIES

%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 10)

# Plotly settings
import plotly.io as pio
pio.templates.default = "plotly_white"

## 1. Load Data

In [ ]:
# Load expenses from database
db = SessionLocal()

expenses = db.query(tables.Expense).join(tables.Category).all()

# Convert to DataFrame
data = []
for expense in expenses:
    data.append({
        'id': expense.id,
        'date': expense.date,
        'merchant': expense.merchant,
        'category': expense.category.name,
        'category_slug': expense.category.slug,
        'amount': expense.amount,
        'payment_method': expense.payment_method,
        'description': expense.description,
        'source': expense.source
    })

df = pd.DataFrame(data)
db.close()

print(f"Total expenses loaded: {len(df)}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Total amount: ${df['amount'].sum():,.2f}")

In [ ]:
# Display first few rows
df.head(10)

In [ ]:
# Data info
df.info()

In [ ]:
# Basic statistics
df.describe()

## 2. Data Cleaning and Preparation

In [ ]:
# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Extract time features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.month_name()
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.day_name()
df['week'] = df['date'].dt.isocalendar().week

# Sort by date
df = df.sort_values('date').reset_index(drop=True)

print("✅ Data cleaned and prepared")

## 3. Exploratory Data Analysis

In [ ]:
# Category distribution
category_counts = df['category'].value_counts()

fig = px.pie(
    values=category_counts.values,
    names=category_counts.index,
    title='Expense Distribution by Category',
    hole=0.4
)
fig.show()

In [ ]:
# Spending by category
category_spending = df.groupby('category')['amount'].sum().sort_values(ascending=False)

fig = px.bar(
    x=category_spending.index,
    y=category_spending.values,
    title='Total Spending by Category',
    labels={'x': 'Category', 'y': 'Amount ($)'},
    text=category_spending.values
)
fig.update_traces(texttemplate='$%{text:.2f}', textposition='outside')
fig.show()

In [ ]:
# Payment method distribution
payment_dist = df['payment_method'].value_counts()

plt.figure(figsize=(10, 6))
payment_dist.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Payment Method Distribution', fontsize=16, fontweight='bold')
plt.xlabel('Payment Method')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 4. Time Series Analysis

In [ ]:
# Daily spending trend
daily_spending = df.groupby(df['date'].dt.date)['amount'].sum()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=daily_spending.index,
    y=daily_spending.values,
    mode='lines+markers',
    name='Daily Spending',
    line=dict(color='royalblue', width=2),
    fill='tonexty'
))

fig.update_layout(
    title='Daily Spending Trend',
    xaxis_title='Date',
    yaxis_title='Amount ($)',
    hovermode='x unified'
)
fig.show()

In [ ]:
# Monthly spending trend
monthly_spending = df.groupby([df['date'].dt.to_period('M')])['amount'].agg(['sum', 'count', 'mean'])
monthly_spending.index = monthly_spending.index.to_timestamp()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly_spending.index,
    y=monthly_spending['sum'],
    name='Total Spending',
    marker_color='lightblue'
))

fig.add_trace(go.Scatter(
    x=monthly_spending.index,
    y=monthly_spending['mean'],
    name='Average per Transaction',
    yaxis='y2',
    line=dict(color='red', width=2)
))

fig.update_layout(
    title='Monthly Spending Analysis',
    xaxis_title='Month',
    yaxis_title='Total Amount ($)',
    yaxis2=dict(title='Average ($)', overlaying='y', side='right')
)
fig.show()

In [ ]:
# Spending by day of week
dow_spending = df.groupby('day_of_week')['amount'].agg(['sum', 'mean', 'count'])

# Order by day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_spending = dow_spending.reindex(day_order)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Total spending by day
dow_spending['sum'].plot(kind='bar', ax=ax1, color='coral', edgecolor='black')
ax1.set_title('Total Spending by Day of Week', fontsize=14, fontweight='bold')
ax1.set_xlabel('Day')
ax1.set_ylabel('Total Amount ($)')
ax1.tick_params(axis='x', rotation=45)

# Average spending by day
dow_spending['mean'].plot(kind='bar', ax=ax2, color='lightgreen', edgecolor='black')
ax2.set_title('Average Spending by Day of Week', fontsize=14, fontweight='bold')
ax2.set_xlabel('Day')
ax2.set_ylabel('Average Amount ($)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 5. Category Deep Dive

In [ ]:
# Spending trend by category over time
category_monthly = df.groupby([df['date'].dt.to_period('M'), 'category'])['amount'].sum().unstack(fill_value=0)
category_monthly.index = category_monthly.index.to_timestamp()

fig = go.Figure()

for category in category_monthly.columns:
    fig.add_trace(go.Scatter(
        x=category_monthly.index,
        y=category_monthly[category],
        mode='lines+markers',
        name=category,
        stackgroup='one'
    ))

fig.update_layout(
    title='Spending Trends by Category Over Time',
    xaxis_title='Month',
    yaxis_title='Amount ($)',
    hovermode='x unified'
)
fig.show()

In [ ]:
# Category statistics
category_stats = df.groupby('category')['amount'].agg([
    ('Total', 'sum'),
    ('Count', 'count'),
    ('Average', 'mean'),
    ('Median', 'median'),
    ('Min', 'min'),
    ('Max', 'max'),
    ('Std', 'std')
]).round(2)

category_stats.sort_values('Total', ascending=False)

## 6. Top Merchants Analysis

In [ ]:
# Top 10 merchants by total spending
top_merchants = df.groupby('merchant')['amount'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False).head(10)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=top_merchants.index,
    y=top_merchants['sum'],
    text=top_merchants['sum'].round(2),
    texttemplate='$%{text}',
    textposition='outside',
    marker_color='lightcoral'
))

fig.update_layout(
    title='Top 10 Merchants by Total Spending',
    xaxis_title='Merchant',
    yaxis_title='Total Amount ($)',
    xaxis_tickangle=-45
)
fig.show()

print("\nDetailed merchant statistics:")
top_merchants

## 7. Anomaly Detection

In [ ]:
# Detect outliers using Z-score
from scipy import stats

df['z_score'] = np.abs(stats.zscore(df['amount']))
anomalies = df[df['z_score'] > 3]

print(f"Found {len(anomalies)} anomalies (Z-score > 3)")
print("\nAnomaly details:")
anomalies[['date', 'merchant', 'category', 'amount', 'z_score']].sort_values('amount', ascending=False)

In [ ]:
# Visualize anomalies
fig = go.Figure()

# Normal expenses
normal = df[df['z_score'] <= 3]
fig.add_trace(go.Scatter(
    x=normal['date'],
    y=normal['amount'],
    mode='markers',
    name='Normal',
    marker=dict(color='blue', size=6, opacity=0.6)
))

# Anomalies
fig.add_trace(go.Scatter(
    x=anomalies['date'],
    y=anomalies['amount'],
    mode='markers',
    name='Anomaly',
    marker=dict(color='red', size=12, symbol='x')
))

fig.update_layout(
    title='Expense Anomaly Detection',
    xaxis_title='Date',
    yaxis_title='Amount ($)',
    hovermode='closest'
)
fig.show()

## 8. Correlation Analysis

In [ ]:
# Create pivot table for correlation
category_pivot = df.pivot_table(
    values='amount',
    index=df['date'].dt.to_period('M'),
    columns='category',
    aggfunc='sum',
    fill_value=0
)

# Calculate correlation
correlation = category_pivot.corr()

# Visualize
plt.figure(figsize=(12, 10))
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Category Spending Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 9. Insights and Recommendations

In [ ]:
# Generate insights
total_spending = df['amount'].sum()
avg_transaction = df['amount'].mean()
most_expensive_category = df.groupby('category')['amount'].sum().idxmax()
most_frequent_category = df['category'].mode()[0]
highest_spending_day = dow_spending['sum'].idxmax()
highest_spending_month = monthly_spending['sum'].idxmax().strftime('%B %Y')

print("📊 KEY INSIGHTS")
print("=" * 60)
print(f"\n💰 Total Spending: ${total_spending:,.2f}")
print(f"📈 Average Transaction: ${avg_transaction:.2f}")
print(f"🏆 Most Expensive Category: {most_expensive_category}")
print(f"📍 Most Frequent Category: {most_frequent_category}")
print(f"📅 Highest Spending Day: {highest_spending_day}")
print(f"🗓️  Highest Spending Month: {highest_spending_month}")
print(f"⚠️  Anomalies Detected: {len(anomalies)}")

print("\n💡 RECOMMENDATIONS")
print("=" * 60)

# Recommendation 1: High spending category
top_category_pct = (df[df['category'] == most_expensive_category]['amount'].sum() / total_spending) * 100
print(f"\n1. Your '{most_expensive_category}' expenses account for {top_category_pct:.1f}% of total spending.")
print(f"   Consider setting a monthly budget for this category.")

# Recommendation 2: Day of week pattern
print(f"\n2. You tend to spend more on {highest_spending_day}s.")
print(f"   Be mindful of expenses on this day.")

# Recommendation 3: Anomalies
if len(anomalies) > 0:
    print(f"\n3. {len(anomalies)} unusual expenses detected.")
    print(f"   Review these transactions for potential savings.")

# Recommendation 4: Payment methods
payment_counts = df['payment_method'].value_counts()
if len(payment_counts) > 1:
    print(f"\n4. Using multiple payment methods ({len(payment_counts)} different methods).")
    print(f"   Consider consolidating to earn more rewards.")

## 10. Export Summary Report

In [ ]:
# Create summary report
summary_report = {
    'Analysis Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Total Expenses': len(df),
    'Date Range': f"{df['date'].min()} to {df['date'].max()}",
    'Total Spending': f"${total_spending:,.2f}",
    'Average Transaction': f"${avg_transaction:.2f}",
    'Most Expensive Category': most_expensive_category,
    'Most Frequent Category': most_frequent_category,
    'Anomalies Detected': len(anomalies),
    'Top Merchant': df.groupby('merchant')['amount'].sum().idxmax()
}

# Convert to DataFrame
summary_df = pd.DataFrame([summary_report]).T
summary_df.columns = ['Value']

print("\n📄 ANALYSIS SUMMARY")
print("=" * 60)
print(summary_df)

# Save to CSV
output_path = Path('../data/processed/analysis_summary.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)
summary_df.to_csv(output_path)
print(f"\n✅ Summary saved to {output_path}")